# SQL 작성

In [1]:
import sys
import os
import pandas as pd
from scipy import io
import cv2
from glob import glob
sys.path.append('../../src')
import DISData as DD

## SQL 연결

In [2]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [3]:
query1 = (f'''CREATE TABLE IF NOT EXISTS DIS_300W_3D_landmark_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(1000) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(1000) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            label VARCHAR(20) NOT NULL,
            landmark VARCHAR(1000));''')

In [4]:
query2 = (f'''INSERT INTO DIS_300W_3D_landmark_info 
        (img_name, img_dir, img_format, img_width, img_height, 
        color_space, label_name, label_dir, label_format, label,
        landmark)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

## qurey 실행

In [5]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [11]:
# label 경로
label_path = 'Z:\\DIS_300W_3D\\data\\RGB\\labeldata'
image_path = 'Z:\\DIS_300W_3D\\data\\RGB\\imagedata'

value_list1 = []
error_list = []
count = 0

label_ids = os.listdir(label_path)
for num, label_id in enumerate(label_ids):
    total_cnt = len(label_ids)
    label_files = os.listdir(os.path.join(label_path,label_id))
    for label_file in label_files:
        # try:
            label = os.path.join(label_path,label_id,label_file)
            text = io.loadmat(label)
            # print(text)
            image = label.replace('labeldata','rawdata').replace('.mat','.jpg')    
            img_dir = image
            img_name, img_format = os.path.splitext(os.path.basename(image))
            img_format = img_format.replace('.','')
            image = cv2.imread(img_dir)
            img_width = image.shape[1]
            img_height = image.shape[0]
            color_check = img_dir.split('\\')[-2]

            color_space = 'RGB'
            label_dir = label
            label_name, label_format = os.path.splitext(os.path.basename(label))
            label_format = label_format.replace('.','')
            label = 'face_68'

            pt2d_x = text['pt2d'][0]
            pt2d_y = text['pt2d'][1]
            

            point = []
            for pt in range(len(pt2d_x)):
                point.append((int(float(pt2d_x[pt])),int(float(pt2d_y[pt]))))

            landmark = ','.join(map(str, [ (x,y) for x,y in point]))

            img_dir = img_dir.replace('Z:', 'DataBase').replace('\\','/')
            label_dir = label_dir.replace('Z:', 'DataBase').replace('\\','/')

            value1 = (img_name, img_dir, img_format, img_width, img_height, color_space, label_name, label_dir, label_format, label, landmark)
            value_list1.append(value1)

        # except Exception as ex:
        #     error_list.append((label, str(ex)))
        #     print(f"Error loading data from file: {label}")
        #     print(f"Error details: {ex}")
            
            if len(value_list1) >= 500:
                # print(value_list1[0])
                doUT.insert_dataset_values(query2, value_list1)
                print(num, total_cnt)
                value_list1 = []

doUT.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')

insert 500 done
1 4
insert 500 done
1 4
insert 500 done
1 4
insert 500 done
1 4
insert 500 done
1 4
insert 500 done
3 4
insert 500 done
3 4
insert 337 done
done


In [26]:
doUT.connect_end()

SQL connection closed
